In [1]:
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib
import traceback

from tqdm import tqdm
import pandas as pd



from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [75]:
!{sys.executable} -m pip install -r requirements.txt

  Using cached pandas-1.2.4-cp37-cp37m-manylinux1_x86_64.whl (9.9 MB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
     |████████████████████████████████| 15.3 MB 763 kB/s eta 0:00:01


In [2]:
class Args():
    def __init__(self, term='restaurants', location='Philadelphia, Pa', streets_path='street_name.txt', data_path= 'data.pkl',iteration=0):
        
        self.term=term
        self.location=location
        self.street_path=street_path
        self.data_path=data_path
        self.iteration=iteration
        
    def init_parsearges(self):
        ap = argparse.ArgumentParser()
        
        # Argument of insightface
        ap.add_argument("--term", default=self.term,
                        type=str, help='Search term')
        
        ap.add_argument("--location", default=self.location,
                        type=str, help='Search location')
        ap.add_argument("--street_path", default=self.street_path,
                        type=str, help='path to names of the streets')
        ap.add_argument("--data_path", default=self.data_path,
                        type=str, help='path to save data')
        
        ap.add_argument("--iteration", default=self.iteration,
                         type=int, help='starting iteration')

        args = ap.parse_args()
        
        return args


# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_KEY= 'cwhlm9INXDobbgkfHlDhVVVVrncPHYNw9DoTcVLXuHkeCcGVVj7F7iPgMqaeW2JH5fKS24dRBQJoIkKJfLLgpJCGbPLmMuHSIqHsFsyyggXyzcohMWKSajdxGOHfX3Yx' 


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'restaurants'
DEFAULT_LOCATION = 'Philadelphia, Pa'
SEARCH_LIMIT = 50


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

#     print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def query_api(term, location):
    """Queries the API by the input values from the user.

    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)
    return response

    businesses = response.get('businesses')

    if not businesses:
        print(response)
        return []

#     business_id = businesses[0]['id']

#     print(u'{0} businesses found, querying business info ' \
#         'for the top result "{1}" ...'.format(
#             len(businesses), business_id))
#     response = get_business(API_KEY, business_id)

#     data = [get_business(API_KEY, business['id']) for business in businesses ]
#     print(u'Result for business "{0}" found:'.format(business_id))
#     pprint.pprint(response, indent=2)
    return businesses

def read_street(path='street_name.txt'):
    with open(path, 'r') as f:
        streets = f.readlines()
    return [street.replace("\n","") for street in streets]

def read_data(path='data.pkl'):
    return pd.read_pickle(path)

def main(term='restaurants', location='Philadelphia, Pa', streets_path='street_name.txt', data_path= 'data.pkl',iteration=0):
    
#     parser = argparse.ArgumentParser()

#     parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM,
#                         type=str, help='Search term (default: %(default)s)')
#     parser.add_argument('-l', '--location', dest='location',
#                         default=DEFAULT_LOCATION, type=str,
#                         help='Search location (default: %(default)s)')

#     input_values = parser.parse_args()


    streets = read_street(path=streets_path)
    data=[] if iteration==0 else read_data(data_path).to_dict(orient='records')
    for (i, street) in tqdm(enumerate(streets)):
        if i>=iteration: 
            address = street +' '+ location
            try:
                data.extend(query_api(term, address))
            except:
                df = pd.DataFrame(data)
                df.to_pickle(data_path)
                traceback.print_exc()
                sys.exit()
        else:
            pass
        
    print("-------------------SAVING DATA---------------------")
    df = pd.DataFrame(data)
    df.to_pickle(data_path)
    
    return df

In [ ]:
args=Args(term='restaurants', location='Philadelphia, Pa', streets_path='street_name.txt', data_path= 'data.pkl',iteration=0)
# args=args.init_parsearges()
b=main(term=args.term, location=args.location, streets_path=args.street_path, data_path= args.data_path,iteration=args.iteration)

# Data Preprocess

In [3]:
df = read_data(path='data.pkl')
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,xmxm5eK2eBfJQkpygVZ5eg,wissahickon-bar-and-bistro-philadelphia,Wissahickon Bar & Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/mQvgif...,False,https://www.yelp.com/biz/wissahickon-bar-and-b...,10,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 40.0171089172363, 'longitude': -7...","[delivery, pickup]","{'address1': '5109 Rochelle Ave', 'address2': ...",+12154828583,(215) 482-8583,412.945173,NaN
1,uIlFxzyoJy7ud9j12aK7MA,dj-kitchen-philadelphia,DJ Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/cK-4LG...,False,https://www.yelp.com/biz/dj-kitchen-philadelph...,102,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.5,"{'latitude': 40.007287, 'longitude': -75.212717}","[delivery, pickup]","{'address1': '4040 City Ave', 'address2': None...",+12155868888,(215) 586-8888,1205.086367,$$
2,Xkdu_37aNqJxilh9XQSmEA,the-rook-manayunk-philadelphia,The Rook Manayunk,https://s3-media4.fl.yelpcdn.com/bphoto/WVScy2...,False,https://www.yelp.com/biz/the-rook-manayunk-phi...,42,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 40.021691, 'longitude': -75.215963}","[delivery, pickup]","{'address1': '4001 Cresson St', 'address2': ''...",+12673232906,(267) 323-2906,1019.677610,$$
3,8QtgPszOI5-4qwizvWi5lA,unity-taqueria-philadelphia,Unity Taqueria,https://s3-media4.fl.yelpcdn.com/bphoto/Fek1vV...,False,https://www.yelp.com/biz/unity-taqueria-philad...,3,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",5.0,"{'latitude': 40.022452, 'longitude': -75.209024}",[delivery],"{'address1': '5420 Ridge Ave', 'address2': Non...",+12158425829,(215) 842-5829,668.481702,NaN
4,eFB8nsciED7EupYVyAnTyA,dekes-bar-b-que-philadelphia-3,Deke's Bar-B-Que,https://s3-media2.fl.yelpcdn.com/bphoto/I4R1CW...,False,https://www.yelp.com/biz/dekes-bar-b-que-phila...,176,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.0,"{'latitude': 40.0152739, 'longitude': -75.2075...","[delivery, pickup]","{'address1': '4901 Ridge Ave', 'address2': '',...",+12155887427,(215) 588-7427,245.909097,$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133218,qrj59uv5by9NUCzLOpeSHQ,new-station-pizza-8-philadelphia,New Station Pizza 8,https://s3-media3.fl.yelpcdn.com/bphoto/6NBtzg...,False,https://www.yelp.com/biz/new-station-pizza-8-p...,5,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 40.01071, 'longitude': -75.1646}","[delivery, pickup]","{'address1': '2206 W Hunting Park Ave', 'addre...",+12152251323,(215) 225-1323,1091.848510,$$
133219,3HtellZPFSOaVNKvMMKycA,wayne-and-berkley-ii-philadelphia,Wayne & Berkley II,https://s3-media2.fl.yelpcdn.com/bphoto/zaIVug...,False,https://www.yelp.com/biz/wayne-and-berkley-ii-...,3,"[{'alias': 'seafood', 'title': 'Seafood'}]",2.5,"{'latitude': 40.00947, 'longitude': -75.16647}",[],"{'address1': '3556 N 23rd St', 'address2': '',...",,,881.134937,NaN
133220,6xQ3Q-kDlbxyB-bIy3cHkg,dou-hueng-philadelphia,Dou Hueng,https://s3-media2.fl.yelpcdn.com/bphoto/28ZytP...,False,https://www.yelp.com/biz/dou-hueng-philadelphi...,1,"[{'alias': 'chinese', 'title': 'Chinese'}]",2.0,"{'latitude': 39.99675, 'longitude': -75.1778999}",[],"{'address1': '2828 W Lehigh Ave', 'address2': ...",+12152282708,(215) 228-2708,1052.078218,NaN
133221,vpV8g_SAY8TR8Jmzevuw5g,subway-philadelphia-96,Subway,https://s3-media2.fl.yelpcdn.com/bphoto/-Na_3q...,False,https://www.yelp.com/biz/subway-philadelphia-9...,7,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",2.0,"{'latitude': 40.0110715054977, 'longitude': -7...","[delivery, pickup]","{'address1': '2920 Robert's Ave', 'address2': ...",+12152211021,(215) 221-1021,552.969653,$


In [4]:
needed_df = df[["name","categories", 'coordinates', 'location']]
needed_df

,name,categories,coordinates,location
0,Wissahickon Bar & Bistro,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...","{'latitude': 40.0171089172363, 'longitude': -7...","{'address1': '5109 Rochelle Ave', 'address2': ..."
1,DJ Kitchen,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...","{'latitude': 40.007287, 'longitude': -75.212717}","{'address1': '4040 City Ave', 'address2': None..."
2,The Rook Manayunk,"[{'alias': 'breakfast_brunch', 'title': 'Break...","{'latitude': 40.021691, 'longitude': -75.215963}","{'address1': '4001 Cresson St', 'address2': ''..."
3,Unity Taqueria,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...","{'latitude': 40.022452, 'longitude': -75.209024}","{'address1': '5420 Ridge Ave', 'address2': Non..."
4,Deke's Bar-B-Que,"[{'alias': 'bbq', 'title': 'Barbeque'}]","{'latitude': 40.0152739, 'longitude': -75.2075...","{'address1': '4901 Ridge Ave', 'address2': '',..."
...,...,...,...,...
133218,New Station Pizza 8,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.01071, 'longitude': -75.1646}","{'address1': '2206 W Hunting Park Ave', 'addre..."
133219,Wayne & Berkley II,"[{'alias': 'seafood', 'title': 'Seafood'}]","{'latitude': 40.00947, 'longitude': -75.16647}","{'address1': '3556 N 23rd St', 'address2': '',..."
133220,Dou Hueng,"[{'alias': 'chinese', 'title': 'Chinese'}]","{'latitude': 39.99675, 'longitude': -75.1778999}","{'address1': '2828 W Lehigh Ave', 'address2': ..."
133221,Subway,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]","{'latitude': 40.0110715054977, 'longitude': -7...","{'address1': '2920 Robert's Ave', 'address2': ..."


In [33]:
needed_df['location'][10]

{'address1': '4213 Ridge Ave',
 'address2': '',
 'address3': None,
 'city': 'Philadelphia',
 'zip_code': '19129',
 'country': 'US',
 'state': 'PA',
 'display_address': ['4213 Ridge Ave', 'Philadelphia, PA 19129']}

In [37]:
df2=pd.DataFrame()
df2['name']=needed_df['name']
df2[['latitude','longtitude']]=needed_df['coordinates'].apply(pd.Series)
df2[['address1','address2','address3','city','zip_code','country','state','display_address']]=needed_df['location'].apply(pd.Series)
df2[['display_address']]=df2['display_address'].apply(foo2)
df2[['alias,title']]=needed_df['categories'].apply(foo)
df2[['alias','title']]=df2['alias,title'].apply(pd.Series)

df2

,name,latitude,longtitude,address1,address2,address3,city,zip_code,country,state,display_address,"alias,title",alias,title
0,Wissahickon Bar & Bistro,40.017109,-75.210449,5109 Rochelle Ave,,None,Philadelphia,19128,US,PA,"5109 Rochelle Ave Philadelphia, PA 19128","{'alias': ['burgers', 'bars'], 'title': ['Burg...","[burgers, bars]","[Burgers, Bars]"
1,DJ Kitchen,40.007287,-75.212717,4040 City Ave,None,None,Philadelphia,19131,US,PA,"4040 City Ave Philadelphia, PA 19131","{'alias': ['chinese', 'bubbletea'], 'title': [...","[chinese, bubbletea]","[Chinese, Bubble Tea]"
2,The Rook Manayunk,40.021691,-75.215963,4001 Cresson St,,None,Philadelphia,19127,US,PA,"4001 Cresson St Philadelphia, PA 19127","{'alias': ['breakfast_brunch', 'juicebars', 's...","[breakfast_brunch, juicebars, sportsbars]","[Breakfast & Brunch, Juice Bars & Smoothies, S..."
3,Unity Taqueria,40.022452,-75.209024,5420 Ridge Ave,None,,Philadelphia,19128,US,PA,"5420 Ridge Ave Philadelphia, PA 19128","{'alias': ['tacos', 'tex-mex'], 'title': ['Tac...","[tacos, tex-mex]","[Tacos, Tex-Mex]"
4,Deke's Bar-B-Que,40.015274,-75.207567,4901 Ridge Ave,,,Philadelphia,19128,US,PA,"4901 Ridge Ave Philadelphia, PA 19128","{'alias': ['bbq'], 'title': ['Barbeque']}",[bbq],[Barbeque]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133218,New Station Pizza 8,40.010710,-75.164600,2206 W Hunting Park Ave,,,Philadelphia,19140,US,PA,"2206 W Hunting Park Ave Philadelphia, PA 19140","{'alias': ['pizza'], 'title': ['Pizza']}",[pizza],[Pizza]
133219,Wayne & Berkley II,40.009470,-75.166470,3556 N 23rd St,,,Philadelphia,19140,US,PA,"3556 N 23rd St Philadelphia, PA 19140","{'alias': ['seafood'], 'title': ['Seafood']}",[seafood],[Seafood]
133220,Dou Hueng,39.996750,-75.177900,2828 W Lehigh Ave,None,,Philadelphia,19132,US,PA,"2828 W Lehigh Ave Philadelphia, PA 19132","{'alias': ['chinese'], 'title': ['Chinese']}",[chinese],[Chinese]
133221,Subway,40.011072,-75.176509,2920 Robert's Ave,Ste 1040,,Philadelphia,19129,US,PA,"2920 Robert's Ave Ste 1040 Philadelphia, PA 19...","{'alias': ['sandwiches'], 'title': ['Sandwiche...",[sandwiches],[Sandwiches]


In [47]:
df3=df2[['name','title','latitude','longtitude','address1','address2','address3','city','zip_code','country','state','display_address']]
df3=df3.drop_duplicates(['name','latitude','longtitude','address1','address2','address3','city','zip_code','country','state','display_address'])
df3.to_csv('data.csv')
df3

,name,title,latitude,longtitude,address1,address2,address3,city,zip_code,country,state,display_address
0,Wissahickon Bar & Bistro,"[Burgers, Bars]",40.017109,-75.210449,5109 Rochelle Ave,,None,Philadelphia,19128,US,PA,"5109 Rochelle Ave Philadelphia, PA 19128"
1,DJ Kitchen,"[Chinese, Bubble Tea]",40.007287,-75.212717,4040 City Ave,None,None,Philadelphia,19131,US,PA,"4040 City Ave Philadelphia, PA 19131"
2,The Rook Manayunk,"[Breakfast & Brunch, Juice Bars & Smoothies, S...",40.021691,-75.215963,4001 Cresson St,,None,Philadelphia,19127,US,PA,"4001 Cresson St Philadelphia, PA 19127"
3,Unity Taqueria,"[Tacos, Tex-Mex]",40.022452,-75.209024,5420 Ridge Ave,None,,Philadelphia,19128,US,PA,"5420 Ridge Ave Philadelphia, PA 19128"
4,Deke's Bar-B-Que,[Barbeque],40.015274,-75.207567,4901 Ridge Ave,,,Philadelphia,19128,US,PA,"4901 Ridge Ave Philadelphia, PA 19128"
...,...,...,...,...,...,...,...,...,...,...,...,...
120921,Panera Bread,"[Sandwiches, Salad, Soup]",40.148937,-74.997931,152 E St Rd,,,Feasterville,19053,US,PA,"152 E St Rd Feasterville, PA 19053"
120925,Pizza Hut,"[Pizza, Italian, Chicken Wings]",40.150030,-75.000600,60 E Street Rd,,None,Feasterville Trevose,19053,US,PA,"60 E Street Rd Feasterville Trevose, PA 19053"
127407,Que Chula Es Puebla,[Mexican],39.972366,-75.139198,1356 N 2nd St,,,Philadelphia,19122,US,PA,"1356 N 2nd St Philadelphia, PA 19122"
127422,El Greco Pizza & Luncheonette,[Pizza],39.973800,-75.138890,1500 N 2nd St,,,Philadelphia,19122,US,PA,"1500 N 2nd St Philadelphia, PA 19122"


In [23]:
a = [len(i) for i in df2['alias,title']]
[print(i) for i in a ]

[]

In [59]:
needed_df['categories'][0]

[{'alias': 'burgers', 'title': 'Burgers'}, {'alias': 'bars', 'title': 'Bars'}]

# Test

In [36]:
def foo(x):
#     print(x)
    dic={}
    for i in x:
#         print(i, i.keys())
        for key in i.keys():
            dic.setdefault(key,[]).append(i[key])
    return dic

def foo2(x):
    string=''
    for i in x:
        string+=i+' '
        
    return string

In [67]:
foo([{'alias': 'burgers', 'title': 'Burgers'}, {'alias': 'bars', 'title': 'Bars'}])

{'alias': 'burgers', 'title': 'Burgers'} dict_keys(['alias', 'title'])
{'alias': 'bars', 'title': 'Bars'} dict_keys(['alias', 'title'])


{'alias': ['burgers', 'bars'], 'title': ['Burgers', 'Bars']}

In [4]:
term='restaurants'
location='Philadelphia, Pa'
r=query_api(term, location)

In [24]:
r.get('businesses')[0]

{'id': '6ajnOk0GcY9xbb5Ocaw8Gw',
 'alias': 'barbuzzo-philadelphia',
 'name': 'Barbuzzo',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/barbuzzo-philadelphia?adjust_creative=LYlilwjYqjn0f0LIANnrQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LYlilwjYqjn0f0LIANnrQg',
 'review_count': 2772,
 'categories': [{'alias': 'mediterranean', 'title': 'Mediterranean'},
  {'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'italian', 'title': 'Italian'}],
 'rating': 4.5,
 'coordinates': {'latitude': 39.94999, 'longitude': -75.16216},
 'transactions': ['pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '110 S 13th St',
  'address2': '',
  'address3': '',
  'city': 'Philadelphia',
  'zip_code': '19107',
  'country': 'US',
  'state': 'PA',
  'display_address': ['110 S 13th St', 'Philadelphia, PA 19107']},
 'phone': '+12155469300',
 'display_phone': '(215) 546-9300',
 'dist

In [136]:
import time
a=[]
a.extend([])
start=time.time()
for i in range(10000):
    a.append(list(range(500)))
end=time.time()
print(end -start)

0.1197962760925293


In [135]:
import time
a=[]
start=time.time()
for i in range(10000):
    a.extend(list(range(500)))
end=time.time()
print(end -start)

0.08768534660339355


In [25]:
a= { 'alias': 'barbuzzo-philadelphia',
  'categories': [ {'alias': 'mediterranean', 'title': 'Mediterranean'},
                  {'alias': 'pizza', 'title': 'Pizza'},
                  {'alias': 'italian', 'title': 'Italian'}],
  'coordinates': {'latitude': 39.94999, 'longitude': -75.16216},
  'display_phone': '(215) 546-9300',
  'hours': [ { 'hours_type': 'REGULAR',
               'is_open_now': False,
               'open': [ { 'day': 0,
                           'end': '2100',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 1,
                           'end': '2100',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 2,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 3,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 4,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1200'},
                         { 'day': 5,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1200'},
                         { 'day': 6,
                           'end': '2100',
                           'is_overnight': False,
                           'start': '1200'}]}],
  'id': '6ajnOk0GcY9xbb5Ocaw8Gw',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
  'is_claimed': True,
  'is_closed': False,
  'location': { 'address1': '110 S 13th St',
                'address2': '',
                'address3': '',
                'city': 'Philadelphia',
                'country': 'US',
                'cross_streets': '',
                'display_address': ['110 S 13th St', 'Philadelphia, PA 19107'],
                'state': 'PA',
                'zip_code': '19107'},
  'name': 'Barbuzzo',
  'phone': '+12155469300',
  'photos': [ 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
              'https://s3-media1.fl.yelpcdn.com/bphoto/dn5Pfj75I1fWZ2LqZ0nJ4Q/o.jpg',
              'https://s3-media1.fl.yelpcdn.com/bphoto/3hdgyod-3e5eW4rfhvVh0A/o.jpg'],
  'price': '$$',
  'rating': 4.5,
  'review_count': 2772,
  'transactions': ['pickup', 'delivery'],
  'url': 'https://www.yelp.com/biz/barbuzzo-philadelphia?adjust_creative=LYlilwjYqjn0f0LIANnrQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=LYlilwjYqjn0f0LIANnrQg'}

b= {'id': '6ajnOk0GcY9xbb5Ocaw8Gw',
 'alias': 'barbuzzo-philadelphia',
 'name': 'Barbuzzo',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/barbuzzo-philadelphia?adjust_creative=LYlilwjYqjn0f0LIANnrQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LYlilwjYqjn0f0LIANnrQg',
 'review_count': 2772,
 'categories': [{'alias': 'mediterranean', 'title': 'Mediterranean'},
  {'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'italian', 'title': 'Italian'}],
 'rating': 4.5,
 'coordinates': {'latitude': 39.94999, 'longitude': -75.16216},
 'transactions': ['pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '110 S 13th St',
  'address2': '',
  'address3': '',
  'city': 'Philadelphia',
  'zip_code': '19107',
  'country': 'US',
  'state': 'PA',
  'display_address': ['110 S 13th St', 'Philadelphia, PA 19107']},
 'phone': '+12155469300',
 'display_phone': '(215) 546-9300',
 'distance': 877.931789849032}